In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from datetime import date, timedelta  
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, linear_model
from sklearn.decomposition import TruncatedSVD
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import unittest
import os



In [ ]:
df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')

#Columns deemed unneccessary for this task
df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1, inplace = True)


In [ ]:
#Filters the code to FX forwards only
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward") | (df['source_instrument_name']).astype(str).str.contains("Forward")]
df = df.drop(['order_request_quotes_received', 'total_value_of_transactions', 'liquidity_provider_flag','executed_transactions', 'median_size_request_quotes_gbp','egb_eligible_flag','data_quality_flag','central_bank_isin_flag', 'data_quality_flag', 'smartie_instrument_name', 'active', 'business', 'order_request_quotes_can_wdrwn','order_request_quotes_modified'], axis = 1)

#df.to_csv('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/RTS27filtered.csv')
print(df['bank_name'].unique())

In [ ]:
df.to_csv('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/RTS27filtered.csv')

In [ ]:
reducedRTS = pd.read_csv("C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/RTS27filtered.csv")

In [ ]:
reducedRTS['date_of_trading_day'] = pd.to_datetime(reducedRTS['date_of_trading_day'])

def get_quarter(start_date, end_date, data):
    mask = (df['date_of_trading_day'] > start_date) & (df['date_of_trading_day'] <= end_date) & (df['source_instrument_name']).astype(str).str.contains("20180321")
    rtsQ = df.loc[mask]
    return rtsQ

#df = df.loc[mask]
q1Rts = get_quarter('2018-03-07', '2018-03-28', df)

fig, ax = plt.subplots(figsize = (12,6))
sns.barplot(x = q1Rts['date_of_trading_day'], y = q1Rts['total_value_of_trans_gbp'], estimator = sum, ci = None ,ax=ax, hue = q1Rts['bank_name'])
x_dates = q1Rts['date_of_trading_day'] .dt.strftime('%Y-%m-%d').sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')
print(q1Rts['bank_name'].unique())

In [ ]:
# Select target
y = df['total_value_of_transactions']
# To keep things simple, we'll use only numerical predictors
dropped_df = df.drop(['total_value_of_transactions'], axis=1)
X = dropped_df

im = SimpleImputer(strategy='mean',missing_values=np.nan)
y = y.to_numpy().reshape(-1,1)
y = im.fit_transform(y)
y = pd.DataFrame(y)
print(y)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

categorical_cols = [col for col in X.columns if X[col].dtype in ['object']]

# Select numerical columns
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
       
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='mean',missing_values=np.nan)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',missing_values=None, fill_value='missing_value')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

sc = StandardScaler()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ], remainder='passthrough')

model = linear_model.LinearRegression()

#print(preprocessor.fit_transform(X_train, y_train))

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('scaler', sc),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)


# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)
svd = TruncatedSVD(n_components=5)
svd.fit_transform(X_train, y_train, mean=False)
svd.transform(preds, mean=False)
print(X_train.head(10))

print('Coefficients: \n', model.coef_)

In [ ]:
fig, ax = plt.subplots(figsize = (12,6)) 
#sns.barplot(x = df['date_of_trading_day'], y = df['total_value_of_trans_eur'], estimator = sum, ci = None, ax=ax, hue = df['bank_name'])

sns.countplot(x = df['date_of_trading_day'], ax=ax, hue = df['bank_name'])

x_dates = df['date_of_trading_day'] .dt.strftime('%Y-%m-%d').sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

#new graph - showing off the total value of gbp traded
volume = df.groupby('total_value_of_trans_gbp')
fig, ax = plt.subplots(figsize = (12,6))
sns.barplot(x = df['date_of_trading_day'], y = df['total_value_of_trans_gbp'], estimator = sum, ci = None ,ax=ax, hue = df['bank_name'])
x_dates = df['date_of_trading_day'] .dt.strftime('%Y-%m-%d').sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

sns.pairplot(reducedRTS, hue = 'bank_name')
plt.show()

In [ ]:
TESTDATA_FILENAME = 'C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.feather'
dfTest = pd.read_feather('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.feather')

In [ ]:
class TestNotebook(unittest.TestCase):
    
    
    def test_add(self):
        self.assertEqual(4,4)

    #checks that the dataframes contain frames
    def test_data_read(self):
        if dfTest.empty:
            self.assertTrue(False)
        else:
            self.assertTrue(True)
    
    #Test to check that columns are disposed of that are not needed        
    def test_dropping_columns(self):
        dropped_df = dfTest.drop(['date_created', 'instrument_identifier'], axis=1)
        Xtest = dropped_df
        count = 0
        count2 = 0
        for col in dfTest.columns:
            count = count + 1
        for col in Xtest.columns:
            count2 = count + 1
        self.assertNotEqual(count, count2)
        
   #Test to check that the train data is larger than the test - test the 80% v 20% difference    
    def test_splitting_data_size(self):
        Xtest = dfTest.drop(['date_created', 'instrument_identifier'], axis=1)
        ytest = dfTest['date_created']
        Xtest_train, Xtest_valid, ytest_train, ytest_valid = train_test_split(Xtest, ytest, train_size=0.8, test_size=0.2, random_state=0)
        if Xtest_train.size > Xtest_valid.size:
            self.assertTrue(True)
        else:
            self.assertTrue(False)
    
    unittest.main(argv=[''], verbosity=3, exit=False)
   